## Download and Import Libraries (requirments file)

In [ ]:
!pip install gliner
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
import pandas as pd
from datasets import Dataset
import tqdm


In [ ]:
from gliner import GLiNER
# pretrained NER
model = GLiNER.from_pretrained("Ihor/gliner-biomed-large-v1.0")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

trainer_state.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

##Dataset

In [ ]:
from datasets import load_dataset
dataset_name = "RJT1990/GeneralThoughtArchive"
dataset = load_dataset(dataset_name, split='train', streaming=True)

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
dataset

IterableDataset({
    features: ['question_id', 'question_url', 'question', 'reference_answer', 'prev_messages', 'model_name', 'model_answer', 'model_reasoning', 'task', 'question_license', 'question_source', 'community_answer_score', 'community_question_score', 'verifier_score'],
    num_shards: 18
})

In [ ]:
bio_term = set()
non_bio_term = set()

In [ ]:
ner_labels = [
    "Biology",
    "Zoology",
    "Botany",
    "Micro biology",
    "DNA and RNA",
    "Evolution",
    "Ecology",
    "Physiology",
    "Anatomy",
    "Taxonomy",
    "Medicine",
    "Health",
    "Disease",
    "Pathology",
    "Pharmacology",
    "Surgery",
    "Nursing",
    "Healthcare Communication",
    "Ophthalmology",
    "Dermatology",
    "Psychiatry",
    "Radiology",
    "Immunology",
    "Epidemiology",
    "Neuroscience",
    "Diagnosis",
    "Treatment"
    "Disorder"
    "Medical Exams",
    'Genetic',
    "Medical",
    'Pediatric',
    'Forensic',
    'Parasitology',
]


In [ ]:
# Track terms
non_bio_term = set()
bio_term = set()

# Iterate over tasks
for text in tqdm.tqdm(dataset['task']):

    if "bio" in text.lower():
      bio_term.add(text)
      print(f"FOUND A BIO TERM: {text}")
      continue

    if text in non_bio_term:
        continue

    if text in bio_term:
        continue  # already validated

    entities = model.predict_entities(text, ner_labels, threshold=0.75)

    if entities == []:
        non_bio_term.add(text)
        continue
    else:
        bio_term.add(text)
        print(f"added{text}")


In [ ]:
# Save good terms to a text file
output_file_path_good = "initial_bio_identified_terms.txt"
with open(output_file_path_good, "w") as outfile:
    for item in bio_term:
        outfile.write(f"{item}\n")

print(f"Saved good terms to {output_file_path_good}")



# Save good terms to a text file
output_file_path_good = "non_bio_term.txt"
with open(output_file_path_good, "w") as outfile:
    for item in non_bio_term:
        outfile.write(f"{item}\n")

print(f"Saved bad terms to {output_file_path_good}")

Saved good terms to initial_bio_identified_terms.txt
Saved bad terms to non_bio_term.txt


##Remove remaining Physics and Chemistry terms

In [ ]:
# Define file paths
input_path = "initial_bio_identified_terms.txt"       # original file
output_path = "initial_bio_identified_terms.txt"  # output file

# Keywords to exclude
exclude_keywords = ["Physics", "Chemistry",'Calculus', 'Chinese', 'Spanish', "Ionization"]

with open(input_path, "r") as infile:
    lines = infile.readlines()

filtered_lines = [
    line for line in lines
    if not any(keyword in line for keyword in exclude_keywords)
]

with open(output_path, "w") as outfile:
    outfile.writelines(filtered_lines)

print(f"Filtered file saved to {output_path}")



Filtered file saved to initial_bio_identified_terms.txt


## Math terms removed

In [ ]:
math_removed_results = []
math_list = []

with open("/content/initial_bio_identified_terms.txt", "r") as infile:
    for line in infile:
        text = line.strip()  # remove \n and spaces

        # If the word/phrase contains "Bio", always send it to removed
        if "Bio" in text:
            math_removed_results.append(text)
            continue

        # Otherwise, run entity prediction
        entities = model.predict_entities(text, ['Math'], threshold=0.81)

        if entities:   # only save if not []
            math_list.append(text)
        else:
            math_removed_results.append(text)

print("Math list:", len(math_list))
print("Math removed list:", len(math_removed_results))


Math list: 329
Math removed list: 604


##Include Biomedical terms from list of non-bio terms

In [ ]:
# sometimes NER misses certain bio medical terms, i reviewed the list and included it

list_of_good_bio_terms = [
    'Medical', 'Genetics', 'Healthcare', 'Clinical',
    'Protein', 'Dermatology', 'Urology', 'Virology',
    'Paleontology', 'Neurology', 'Hematology', 'Ophthalmology', 'NHS QA'
]

with open("non_bio_term.txt", "r") as infile:
    for line in infile:
        text = line.strip()  # remove \n and spaces
        for good_term in list_of_good_bio_terms:
            if good_term.lower() in text.lower():
                print(f'####### {text} (matched {good_term})')
                math_removed_results.append(text)


####### Medical Terminology (matched Medical)
####### Medical Evaluation (matched Medical)
####### Medical Law Problem Question (matched Medical)
####### Medical Exams (matched Medical)
####### Clinical Study Design (matched Clinical)
####### Ophthalmology (matched Ophthalmology)
####### Medical Physics (matched Medical)
####### Paleontology (matched Paleontology)
####### Medical Decision Making (matched Medical)
####### Healthcare History (matched Healthcare)
####### Population Genetics (matched Genetics)
####### Medical Ethics (matched Medical)
####### Clinical Signs (matched Clinical)
####### Medical Law Evaluation (matched Medical)
####### Neurology (matched Urology)
####### Neurology (matched Neurology)
####### Medical Statistics (matched Medical)
####### Medical Advice (matched Medical)
####### Protein Folding (matched Protein)
####### Medical Genetics (matched Medical)
####### Medical Genetics (matched Genetics)
####### Legal Analysis - Clinical Negligence (matched Clinical)
###

In [ ]:
# Save the math_removed_results list to a text file
output_file_path = "Final_list_bio_terms.txt"
with open(output_file_path, "w") as outfile:
    for item in math_removed_results:
        outfile.write(f"{item}\n")

print(f"Saved good bio terms to {output_file_path}")

Saved good bio terms to Final_list_bio_terms.txt


##Read the contents inside the file & get the HF data on selected columns


In [ ]:
keywords = []
with open('/content/Final_list_bio_terms.txt', 'r') as f:
    for line in f:
        keywords.append(line.strip())


In [ ]:
def check_task_for_keywords(row):
    task = row['task']
    for keyword in keywords:
        if keyword.lower() in task.lower():
            return True
    return False

filtered_dataset = dataset.filter(check_task_for_keywords)
print(filtered_dataset)

In [ ]:
columns_to_keep = ['question_id', 'question', 'model_answer', 'model_reasoning', 'task']
selected_columns_dataset = filtered_dataset.select_columns(columns_to_keep)
print(selected_columns_dataset)


# keep only:

  # question_id
  # question
  # model answer
  # model_reasoning
  # task
  #... can be adjusted

In [ ]:

if not isinstance(selected_columns_dataset, Dataset):
    selected_columns_dataset = Dataset.from_dict(selected_columns_dataset)

print(selected_columns_dataset)